In [18]:
import numpy as np
import cv2 as cv 
import os


print("NumPy version:", np.__version__)
print("OpenCV version:", cv.__version__)

import sys
print("Python/OS module version:", sys.version)

NumPy version: 2.1.2
OpenCV version: 4.10.0
Python/OS module version: 3.13.0 | packaged by conda-forge | (main, Oct  8 2024, 19:54:55) [MSC v.1941 64 bit (AMD64)]


### `load_templates` Function

This function loads grayscale images from a specified directory. These images are treated as templates and are used for pattern matching in the second task.

In [19]:
def load_templates(path):
    
    templates = {}
    
    if not os.path.exists(path):
        print(f"Error: Templates path '{path}' not found")
        return templates
    
    
    files = os.listdir(path)
    for file in files:
        if file.lower().endswith('.jpg'):
            file_path = os.path.join(path,file)
            image = cv.imread(file_path,cv.IMREAD_GRAYSCALE)
            
            if image is None:
                print(f"Error loading image '{file}'. Skipping...")
                continue
            
            name = file.split('.')[0]
            templates[name] = image
            
    return templates             

### `load_turn_data` Function
This function loads a file containing turn data for a game. It processes each line in the file, extracting the player's name and the round in which they started playing. The function stores this information in a dictionary, where the keys are the round numbers (converted to integers) and the values are the corresponding player names. 

In [20]:
def load_turn_data(input_folder, turn_filename):
    turns_data = {}
    with open(os.path.join(input_folder, turn_filename), 'r') as f:
        for line in f:
            player, round_start = line.strip().split()
            turns_data[int(round_start)] = player
    return turns_data

### `load_images` Function
This fuction loads two consecutive images for further comparison

In [21]:
def load_images(k, game_files, input_folder, auxiliary_images):
    if k == 0:
        img1 = cv.imread(os.path.join(auxiliary_images, '01.jpg'))
        img2 = cv.imread(os.path.join(input_folder, game_files[k]))
    else:
        img1 = cv.imread(os.path.join(input_folder, game_files[k-1]))
        img2 = cv.imread(os.path.join(input_folder, game_files[k]))
    return img1, img2

### `initialize_matrix` Function

In [22]:
def initialize_matrix():
    coordonate_3x = [(0,0),(0,6),(0,7),(0,13),(6,0),(6,13),(7,0),(7,13),(13,0),(13,6),(13,7),(13,13)]
    coordonate_2x = [(1,1),(1,12),(2,2),(2,11),(3,3),(3,10),(4,4),(4,9),(9,4),(9,9),(10,3),(10,10),(11,2),(11,11),(12,1),(12,12)]
    coordonate_adunare = [(3,6),(4,7),(6,4),(7,3),(6,10),(7,9),(9,6),(10,7)]
    coordonate_inmultire = [(3,7),(4,6),(6,3),(7,4),(6,9),(7,10),(9,7),(10,6)]
    coordonate_impartire = [(1,4),(1,9),(4,1),(9,1),(4,12),(9,12),(12,4),(12,9)]
    coordonate_scadere = [(2,5),(2,8),(5,2),(5,11),(8,2),(8,11),(11,5),(11,8)]
    
    # Initialize matrix with "empty" value
    matrix = np.full((14, 14), 'empty', dtype=object)
    
    for coord in coordonate_3x:
        matrix[coord[0],coord[1]] = '3x'
    for coord in coordonate_2x:
        matrix[coord[0],coord[1]] = '2x'
    for coord in coordonate_adunare:
        matrix[coord[0],coord[1]] = '+'
    for coord in coordonate_inmultire:
        matrix[coord[0],coord[1]] = 'x'
    for coord in coordonate_impartire:
        matrix[coord[0],coord[1]] = '/'
    for coord in coordonate_scadere:
        matrix[coord[0],coord[1]] = '-'
    
    matrix[6,6] = 1
    matrix[6,7] = 2
    matrix[7,6] = 3
    matrix[7,7] = 4
    
    return matrix

### `extract_grid` Function
This function is used to extract a specific square region (the game grid) from an input image. It applies several image processing techniques to isolate the blue game grid from the rest of the image, based on the color properties and geometric features of the grid. The function resizes, sharpens, and converts the image to HSV color space to define a mask that isolates the grid. Then, it uses contour detection to extract the rectangular region that corresponds to the game grid. Finally, the function crops and resizes the extracted grid to a fixed size suitable for further processing.

In [23]:
def extract_grid(image, game_nr, k):
    image = cv.resize(image, (0, 0), fx=0.5, fy=0.5)

    # Image preprocessing (blurring and sharpening)
    image_m_blur = cv.medianBlur(image, 3)
    image_g_blur = cv.GaussianBlur(image_m_blur, (0, 0), 5)
    image_sharpened = cv.addWeighted(image_m_blur, 3, image_g_blur, -1.3, 0)  
    
    
    # SAVE SHARPENED IMAGE
    if not os.path.exists('images_processing_test/1_sharpened_image'):
        os.makedirs('images_processing_test/1_sharpened_image')
    cv.imwrite(os.path.join('images_processing_test/1_sharpened_image', f"{game_nr}_{k+1:02d}.jpg"), image_sharpened)
   

    # HSV Color space conversion
    image_hsv = cv.cvtColor(image_sharpened, cv.COLOR_BGR2HSV)
    
    # Color masking (blue color)
    low = (50, 110, 246)
    high = (255, 255, 255)
    mask = cv.inRange(image_hsv, low, high)
    
    
    
    # SAVE MASKED IMAGE
    if not os.path.exists('images_processing_test/2_masked_image'):
        os.makedirs('images_processing_test/2_masked_image')
    cv.imwrite(os.path.join('images_processing_test/2_masked_image', f"{game_nr}_{k+1:02d}.jpg"), mask)
   

    # Mask post-processing (cleaning using erosion)
    kernel = np.ones((5, 5), np.uint8)
    mask = cv.erode(mask, kernel, iterations=1)
    
    
    # SAVE ERODED MASKED IMAGE
    if not os.path.exists('images_processing_test/3_eroded_masked_image'):
        os.makedirs('images_processing_test/3_eroded_masked_image')
    cv.imwrite(os.path.join('images_processing_test/3_eroded_masked_image', f"{game_nr}_{k+1:02d}.jpg"), mask)
   
    
    # Contour detection and extraction
    coords = cv.findNonZero(mask) # Find all non-zero points (text)
    if coords is not None:
        # Find the bounding rectangle that contains all white points from the mask (game board)
        x, y, w, h = cv.boundingRect(coords) 
        
        # Cropping the grid area using the bounding box coordinates
        crop_x_start = x + int(w * 0.127)  # 13% 
        crop_y_start = y + int(h * 0.13)
        crop_x_end = x + int(w * 0.87)    # 87% 
        crop_y_end = y + int(h * 0.875)

        # Cropping the image
        grid = image[crop_y_start:crop_y_end, crop_x_start:crop_x_end]
        
        # Resizing the grid to a fixed size ensuring it contains a 14x14 grid of cells, each 90x90 pixels
        grid = cv.resize(grid, (1260, 1260))
        
    else:
        print(f"Error: Couldn't find any contours in the image '{image}' to extract the grid")
        grid = None
    
    
    # SAVE ERODED MASKED IMAGE
    if not os.path.exists('images_processing_test/4_grid_extracted'):
        os.makedirs('images_processing_test/4_grid_extracted')
    cv.imwrite(os.path.join('images_processing_test/4_grid_extracted', f"{game_nr}_{k+1:02d}.jpg"), grid)
   
    return grid

### `generate_lines` Function
Generate horizontal and vertical lines for a grid based on the given step size and image size.
    
Args:  
- step_size (int): The spacing between the lines (default is 90).  
- image_size (int): The size of the image (default is 1260).  
    
Returns:  
- lines_horizontal (list): List of horizontal lines defined by two points.  
- lines_vertical (list): List of vertical lines defined by two points.  



In [24]:
def generate_lines(step_size=90,image_size=1260):
    lines_horizontal=[] 
    for i in range(0, image_size + 1, step_size):
        l=[]
        l.append((0,i)) 
        l.append((image_size-1,i)) 
        lines_horizontal.append(l)
    
    
    lines_vertical=[]
    for i in range(0, image_size + 1, step_size):
        l=[]
        l.append((i,0))
        l.append((i,image_size-1))
        lines_vertical.append(l)
        
    return lines_horizontal,lines_vertical

### `classify_digit` Function
Classifies the digit in the given patch by comparing it with template images.
    
Args:  
- patch: The cropped patch image containing a digit to be classified.  
- templates: A dictionary of template images, where the key is the template name and the value is the template image.  
  
Returns:  
- token: The name of the template that best matches the patch.  

In [25]:
def classify_digit(patch, templates):
    maxi=-np.inf
    token = None
    for name, img_template in templates.items():
        patch = cv.resize(patch, (img_template.shape[1], img_template.shape[0]))
        corr = cv.matchTemplate(patch,img_template,cv.TM_CCOEFF_NORMED)
        corr=np.max(corr)
        if corr>maxi:
            maxi=corr
            token = name
    return token

### `multiple_equations` Function
Function that checks if the piece placed at a specific position completes multiple equations, taking into account the constraints imposed by the cell where the piece is placed

In [26]:
def multiple_equations(matrix, position, token):
   
    x, y = position
    token = int(token)
    num_equations = 0  # Counter for the number of equations found
    visited_equations = set()  # A set to store the equations we have already found (used to avoid duplicates)
    
    # List the directions: (dx, dy) for up, down, left, right
    directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]
    operators = {'+': lambda a, b: a + b,
                 '-': lambda a, b: a - b,
                 'x': lambda a, b: a * b,
                 '/': lambda a, b: a // b if b != 0 and a % b == 0 else None}
    
    # Check if the current cell has a constraint
    cell_constraint = matrix[position] if matrix[position] in operators else None
    ###print(f"Position: {position}, Token: {token}, Constraint: {cell_constraint}\n")
    
    for dx, dy in directions:
        # Determine the position of the first neighboring cell
        cell_1_positions = (x + dx, y + dy)
        if not (0 <= cell_1_positions[0] < 14 and 0 <= cell_1_positions[1] < 14):
            continue  # Skip if we go out of bounds
        cell_1 = matrix[cell_1_positions]
        if not isinstance(cell_1, int):  # Check if the neighbor is a valid cell (contains a number)
            continue
        
        # Determine the position of the second neighboring cell
        cell_2_positions = (x + 2 * dx, y + 2 * dy)
        if not (0 <= cell_2_positions[0] < 14 and 0 <= cell_2_positions[1] < 14):
            continue  # Skip if we go out of bounds
        cell_2 = matrix[cell_2_positions]
        if not isinstance(cell_2, int): # Check if the neighbor is a valid cell (contains a number)
            continue
        
        for operator, func in operators.items():
            if cell_constraint is not None and cell_constraint != operator:
                continue  # Skip if the current cell cointains a constraint that is not satisfied
            result = func(cell_1, cell_2)
            if result == token:
                equation_id = (cell_1_positions, cell_2_positions)
                if equation_id not in visited_equations:
                    visited_equations.add(equation_id)
                    num_equations += 1
                    ###print(f"Equation added: {cell_1} {operator} {cell_2} = {result}. ID: {equation_id}\n")
            
            
            result_reverse = func(cell_2, cell_1)
            if result_reverse == token:
                equation_id = (cell_1_positions, cell_2_positions)
                if equation_id not in visited_equations:
                    visited_equations.add(equation_id)
                    num_equations += 1
                    ###print(f"Equation added (reversed): {cell_2} {operator} {cell_1} = {result_reverse}. ID: {equation_id}\n")
                    
    ###print(f"Total equations found: {num_equations}\n")
    return num_equations


### `detect_bonus` Functions
Gets the cell type/content at the given position for bonus

In [27]:
def detect_bonus(matrix, position):
    
    cell = matrix[position]
    
    # Check the type of the cell and return the corresponding bonus
    if cell == '3x':
        return 3  
    elif cell == '2x':
        return 2  
    else:
        return 1  # No bonus


### `calculate_move_score` Function
Calculates the total score corresponding to a move made by a player

In [28]:
def calculate_move_score(matrix, position, token):
    base_score = int(token)
    bonus = detect_bonus(matrix, position)
    no_equations = multiple_equations(matrix, position, token)
    
    return base_score * bonus * no_equations

### `detect_and_extract_piece` Function
Detects and extracts a piece (patch) from two images based on the absolute difference. Extracts the patch with the highest pixel intensity mean from the difference image.
- img1 (numpy.ndarray): First image to compare.
- img2 (numpy.ndarray): Second image to compare.
- patch (numpy.ndarray): Extracted patch based on the detected difference.



In [29]:
def detect_and_extract_piece(img1,img2,k,game_nr):
    img1 = extract_grid(img1, game_nr, k)
    img2 = extract_grid(img2, game_nr, k)
        
    # Calculăm diferența absolută
    difference_bgr = cv.absdiff(img1, img2)
    difference_gray = cv.cvtColor(difference_bgr, cv.COLOR_BGR2GRAY)
    
    
    
    # SAVE DIFFERENCE IMAGE
    if not os.path.exists('images_processing_test/5_difference_image'):
        os.makedirs('images_processing_test/5_difference_image')
    cv.imwrite(os.path.join('images_processing_test/5_difference_image', f"{game_nr}_{k+1:02d}.jpg"), difference_gray)
   
    
    # Initialize variables
    max_mean = -np.inf
    patch_coords_matrix = None
    x1, x2, y1, y2 = 0, 0, 0, 0  # Initialize coordinates of the patch
    lines_horizontal, lines_vertical = generate_lines()
    
    # Iterate through each cell in the difference image to find the one with the maximum mean intensity
    for i in range(len(lines_horizontal) - 1):
        for j in range(len(lines_vertical) - 1):
            y_min = lines_vertical[j][0][0]
            y_max = lines_vertical[j + 1][1][0]
            x_min = lines_horizontal[i][0][1]
            x_max = lines_horizontal[i + 1][1][1]
            patch = difference_gray[x_min:x_max, y_min:y_max].copy()
            patch_mean = np.mean(patch)
            
            if patch_mean > max_mean:
                max_mean = patch_mean
                patch_coords_matrix = (i, j)
                x1,x2,y1,y2 = x_min,x_max, y_min,y_max
                

    patch = img2[x1:x2, y1:y2].copy()
    
    
    
    # SAVE DETECTED PATCH
    if not os.path.exists('images_processing_test/6_detected_patch'):
        os.makedirs('images_processing_test/6_detected_patch')
    cv.imwrite(os.path.join('images_processing_test/6_detected_patch', f"{game_nr}_{k+1:02d}.jpg"), patch) 
   
   

    return patch_coords_matrix, patch
        

### `detect_digits` Function
This function processes an image patch to detect and classify digits using predefined templates. The function works by performing several key image processing steps to isolate and identify the digits within a given patch. These steps include cropping the patch, converting it to grayscale, applying binary thresholding, and detecting contours to locate potential digit regions. Once potential digits are identified, the function classifies each digit using a template matching technique.

In [30]:
def detect_digits(patch, templates, k, game_nr):
    # Crop the patch to remove potential noise
    cropped_patch = patch[10:-10, 10:-10].copy()
        
    # Convert the cropped patch to grayscale if it has 3 channels (BGR)
    if len(cropped_patch.shape) == 3: 
        cropped_patch = cv.cvtColor(cropped_patch, cv.COLOR_BGR2GRAY)
        
    
    
    # SAVE CROPPED PATCH
    if not os.path.exists('images_processing_test/7_cropped_patch'):
        os.makedirs('images_processing_test/7_cropped_patch')
    cv.imwrite(os.path.join('images_processing_test/7_cropped_patch', f"{game_nr}_{k+1:02d}.jpg"), cropped_patch)
   
        
    # Apply binary thresholding to the cropped patch
    _, binary_patch = cv.threshold(cropped_patch, 120, 255, cv.THRESH_BINARY)
    binary_patch = cv.bitwise_not(binary_patch)
        
    
    # SAVE BINARY PATCH
    if not os.path.exists('images_processing_test/8_binary_patch'):
        os.makedirs('images_processing_test/8_binary_patch')
    cv.imwrite(os.path.join('images_processing_test/8_binary_patch', f"{game_nr}_{k+1:02d}.jpg"), binary_patch)
   
        
    # Detect contours in the binary image and filter out small contours.
    contours, _ = cv.findContours(binary_patch, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

    digit_regions = []
    for contour in contours:
        # Get the bounding box coordinates of each contour
        x, y, w, h = cv.boundingRect(contour) 
        # Filter out small contours to remove noise and false detections
        if h > 20 and w > 10:  
            digit_regions.append((x, y, w, h))
    # Sort the digit regions by their x-coordinate to ensure the correct order of digits in the image
    digit_regions = sorted(digit_regions, key=lambda region: region[0]) 
    
    
    patch_copy = cropped_patch.copy()
    for x, y, w, h in digit_regions:
        cv.rectangle(patch_copy, (x, y), (x + w, y + h), (255, 0,0), 2)
    
    
    # SAVE DETECTED DIGITS REGIONS
    if not os.path.exists('images_processing_test/9_detected_digits'):
        os.makedirs('images_processing_test/9_detected_digits')
    cv.imwrite(os.path.join('images_processing_test/9_detected_digits', f"{game_nr}_{k+1:02d}.jpg"), patch_copy)
   
    
    # Classify the digits in the cropped patch using the templates
    detected_digits = []
    for x, y, w, h in digit_regions:
        digit = cropped_patch[y:y+h, x:x+w].copy()
        digit_token = classify_digit(digit, templates)
        detected_digits.append(digit_token)
    
    return detected_digits
    
    

### `process_game` function

In [31]:
def process_game(game_nr, game_files, input_folder, output_folder, templates, turns, auxiliary_images):
    
    # Sort game files by round number
    game_files =  sorted(game_files, key=lambda x: int(x.split('_')[1].split('.')[0]))
        
    # Find the corresponding turn file for the current game
    matching_turns = [turn for turn in turns if int(turn.split('_')[0]) == game_nr]
    
    if not matching_turns:
        raise ValueError(f"Turn file for game {game_nr} not found. Check the input folder.")
    
    turn_filename = matching_turns[0]
    
    # Load turn data from the corresponding file
    turns_data = load_turn_data(input_folder, turn_filename)
    sorted_turns = sorted(turns_data.items())
    
    # Initialize game state variables (data matrix and player scores)
    matrix = initialize_matrix()
    player_scores = {player: [] for player in set(turns_data.values())}
    
    current_player = None
    current_round = 0
    current_score = 0
    
    for k in range(len(game_files)):
        img1, img2 = load_images(k, game_files, input_folder, auxiliary_images)
        patch_coords_matrix, patch = detect_and_extract_piece(img1, img2, k, game_nr)
        detected_digits = detect_digits(patch, templates, k, game_nr)
        
        if len(detected_digits) > 1:
            token = detected_digits[0] + detected_digits[1]
        else :
            token = detected_digits[0]
        
        #print(f"Round {k + 1}: Detected position {patch_coords_matrix}, token {token}\n")
        
        
        # Check if the current round is the start of a new round
        for round_start, player in sorted_turns:
            if k + 1 == round_start:
                # Save the score from the previous round
                if current_player is not None:
                    # print(f"End of round {current_round}: Player {current_player}, score {current_score}\n")
                    player_scores[current_player].append((current_round, current_score))
                
                # Start a new round
                current_player = player
                current_round = round_start
                current_score = 0
        
        # Calculate the score for the current move
        score = calculate_move_score(matrix, patch_coords_matrix, token)
        # print(f"Move score: {score}\n")
        current_score += score
        
        # Update the matrix with the new token
        matrix[patch_coords_matrix] = int(token)
    
        # Save the move to an output text file (TASK 1 & 2 SOLUTION)
        output_filename = os.path.join(output_folder, f"{game_nr}_{k+1:02d}.txt")
        with open(output_filename, 'w') as f:
            f.write(f'{patch_coords_matrix[0]+1}{chr(64 + patch_coords_matrix[1]+1)} {token}')
            
    # Save the final score for the last round
    if current_player is not None:
        player_scores[current_player].append((current_round, current_score))
    
    #print(f"Final scores: {player_scores}")
    
    
    all_scores = []
    for player, scores in player_scores.items():
        for round_start, score in scores:
            all_scores.append((player, round_start, score))
    sorted_scores = sorted(all_scores, key=lambda x: x[1])
    #print(sorted_scores)


    # Save the final scores to an output text file (TASK 3 SOLUTION)
    output_file = os.path.join(output_folder, f"{game_nr}_scores.txt")
    content = "\n".join(f"{player} {round_start} {score}" for player, round_start, score in sorted_scores)
    with open(output_file, 'w') as f:
        f.write(content)    
        
    

# Main function of the project
Args:   
    - **input_folder (str)**: Path to the input folder containing game files.  
    - **output_folder (str)**: Path to the output folder where processed files will be saved.  
    - **templates_folder (str)**: Path to the folder containing template files.  
    - **auxiliary_images (str)**: Path to the folder containing auxiliary images.  

In [32]:
def main(input_folder, output_folder, templates_folder, auxiliary_images):
    
    # Load templates
    templates = load_templates(templates_folder)
    
    # Read input data
    try:
        files = os.listdir(input_folder)
    except FileNotFoundError:
        print(f"Error: Input folder '{input_folder}' does not exist.")
        return
    
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    turns = []
    games = {}
    
    # Organize files by game
    for file in files:
        if file.endswith('.txt'):
            if file.endswith('_turns.txt'):
                turns.append(file)
            continue
        file_name = os.path.basename(file)
        try:
            game_nr = int(file_name.split('_')[0])  # Extract game number
        except ValueError:
            print(f"Skipping file '{file}': Cannot extract game number.")
            continue
        if game_nr not in games:
            games[game_nr] = []
        games[game_nr].append(file)
    turns = sorted(turns, key=lambda x: int(x.split('_')[0]))
    
    # Process each game
    for game_nr, game_files in games.items():
        process_game(game_nr, game_files, input_folder, output_folder, templates, turns, auxiliary_images)


### Parameters Settings

In [33]:
# Parameters
input_folder='../testare'
output_folder='341_Lungu_Laura_output_files'
templates_folder = 'templates'
auxiliary_images = '../imagini_auxiliare' 

main(input_folder, output_folder, templates_folder, auxiliary_images)    

# Funcția de evaluare

In [34]:
""" def compare_annotations_position_token(filename_predicted,filename_gt,verbose=0):
	p = open(filename_predicted,"rt")  	
	gt = open(filename_gt,"rt") 

  
	line_gt = gt.readline()
	current_pos_gt, current_token_gt = line_gt.split()
	if verbose:
		print(current_pos_gt,current_token_gt)
	
	line_p = p.readline()
	current_pos_p, current_token_p = line_p.split()
	if verbose:
		print(current_pos_p,current_token_p)
	
	match_positions = 1
	match_tokens = 1
	if(current_pos_p != current_pos_gt):
		match_positions = 0
	if(current_token_p != current_token_gt):
		match_tokens = 0
	

	points_positions = 0.025 * match_positions
	points_tokens = 0.01 * match_tokens	


	return points_positions, points_tokens

#change this on your machine pointing to your results (txt files)
predictions_path_root = "341_Lungu_Laura_output_files/"

#change this on your machine to point to the ground-truth test
#gt_path_root = "../evaluare/cod_evaluare/fake_test_gt/"
gt_path_root = "../evaluare/antrenare_evaluare/"



def compare_annotations_score(filename_predicted,filename_gt,verbose=0):
	p = open(filename_predicted,"rt")  	
	gt = open(filename_gt,"rt") 

	all_lines_gt = gt.readlines()
	all_lines_p = p.readlines()


	number_lines_gt = len(all_lines_gt)

	points_scores = 0

	try:
		for i in range(0,number_lines_gt):
			line_gt = all_lines_gt[i]
			line_p = all_lines_p[i]
			if (line_gt==line_p):
				points_scores += 0.04
	except:
		pass

	return points_scores


#change this to 1 if you want to print results at each turn
verbose = 1
total_points = 0
for game in range(1,5):
# change this for game in range(1,5):
	points_score = 0
	for turn in range(1,51):
		
		name_turn = str(turn)
		if(turn < 10):
			name_turn = '0'+str(turn)

		filename_predicted = predictions_path_root + str(game) + '_' + name_turn + '.txt'
		filename_gt = gt_path_root + str(game) + '_' + name_turn + '.txt'

		game_turn = str(game) + '_' + name_turn
		points_position = 0
		points_tokens = 0	

		try:
			points_position, points_tokens  = compare_annotations_position_token(filename_predicted,filename_gt,verbose)
		except:
			print("Pentru imaginea: ", game_turn, " functia de evaluare a intampinat o eroare")

		print("Imaginea: ", game_turn, "Puncte task 1: ", points_position, "Puncte task 2: ",points_tokens)
		
		total_points = total_points + points_position + points_tokens

	filename_predicted_scores = predictions_path_root + str(game) + '_scores' + '.txt'
	filename_gt_scores = gt_path_root + str(game) + '_scores' + '.txt'
	points_score = compare_annotations_score(filename_predicted_scores,filename_gt_scores,verbose=0)
	print("Puncte task 3 ", points_score)

	total_points = total_points + points_score
	print("Total puncte = ", total_points)


print(total_points) """

' def compare_annotations_position_token(filename_predicted,filename_gt,verbose=0):\n\tp = open(filename_predicted,"rt")  \t\n\tgt = open(filename_gt,"rt") \n\n  \n\tline_gt = gt.readline()\n\tcurrent_pos_gt, current_token_gt = line_gt.split()\n\tif verbose:\n\t\tprint(current_pos_gt,current_token_gt)\n\t\n\tline_p = p.readline()\n\tcurrent_pos_p, current_token_p = line_p.split()\n\tif verbose:\n\t\tprint(current_pos_p,current_token_p)\n\t\n\tmatch_positions = 1\n\tmatch_tokens = 1\n\tif(current_pos_p != current_pos_gt):\n\t\tmatch_positions = 0\n\tif(current_token_p != current_token_gt):\n\t\tmatch_tokens = 0\n\t\n\n\tpoints_positions = 0.025 * match_positions\n\tpoints_tokens = 0.01 * match_tokens\t\n\n\n\treturn points_positions, points_tokens\n\n#change this on your machine pointing to your results (txt files)\npredictions_path_root = "341_Lungu_Laura_output_files/"\n\n#change this on your machine to point to the ground-truth test\n#gt_path_root = "../evaluare/cod_evaluare/fake_tes